In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
# Split the dataset into features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test_scaled)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.8

#Heart Failure Prediction Model

##Loading Data

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the data
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
data.head()

ModuleNotFoundError: No module named 'pandas'

##Splitting Data

In [ ]:
# Separate features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##Standardize the features

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##Define hyperparameter grid for Logistic Regression

In [ ]:

# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs', 'sag', 'saga']
}

# Create a GridSearchCV object with LogisticRegression and optimized scoring
log_reg_cv = GridSearchCV(LogisticRegression(random_state=42), param_grid=param_grid, cv=5, scoring='roc_auc')


##Fitting Model

In [ ]:


# Fit the model on the training data with cross-validation
log_reg_cv.fit(X_train_scaled, y_train)

# Get the best model with optimized hyperparameters
best_model = log_reg_cv.best_estimator_


##Predicting Using Model

In [ ]:
# Predict on the test set using the best model
y_pred = best_model.predict(X_test_scaled)


##Evaluting Model

In [ ]:

# Evaluate model performance with multiple metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

##Saving The Model

In [36]:
import joblib

# Save the trained model
joblib.dump(log_reg_cv.best_estimator_, 'heart_failure_model.pkl')

['heart_failure_model.pkl']

##Loading The model

In [ ]:
# Load the saved model
loaded_model = joblib.load('heart_failure_model.pkl')

# Prepare new data for prediction (ensure it has the same features as the training data)
new_data = ...  # Your new data

# Make predictions
predictions = loaded_model.predict(new_data)